In [10]:
%pip install selenium==4.23.1 webdriver-manager==4.0.2

  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------  9.4/9.4 MB 73.4 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 42.1 MB/s eta 0:00:00
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)

   -

In [11]:
# === Imports & 기본 설정 ===
import os, time, random, re, html
import pandas as pd
from urllib.parse import quote_plus, urljoin

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

INPUT_CSV_PATH = "input.csv"                 # 입력 CSV 경로 (name 컬럼 포함)
NAME_COLUMN = "name"

TEST_OUTPUT_CSV_PATH = "test_output_with_images.csv"
FULL_OUTPUT_CSV_PATH = "output_with_images.csv"

BASE_URL = "https://www.bysuco.com/"
SEARCH_URL_TMPL = "https://www.bysuco.com/product?num=60&page=1&orderBy=popular&keyword={keyword}&kind=bt"

HEADLESS = True                              # 필요 시 False로 바꿔서 눈으로 확인
NAV_TIMEOUT = 25                             # 초, 페이지 로딩/요소 대기 타임아웃
CRAWL_DELAY_RANGE = (0.8, 1.5)               # 요청 사이 랜덤 지연

DEBUG_DIR = "debug_html"
os.makedirs(DEBUG_DIR, exist_ok=True)
SAVE_FIRST_FAILURE_HTML = True


In [12]:
# === Chrome WebDriver ===
def make_driver(headless: bool = True) -> webdriver.Chrome:
    chrome_options = Options()
    if headless:
        # Chrome 109+ headless 모드
        chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1280,2000")
    chrome_options.add_argument("--lang=ko-KR")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0.0.0 Safari/537.36"
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.set_page_load_timeout(NAV_TIMEOUT)
    return driver


# === 공통 유틸 ===
def build_search_url(name: str) -> str:
    return SEARCH_URL_TMPL.format(keyword=quote_plus(name.strip()))

def strip_query(u: str) -> str:
    try:
        return u.split("?", 1)[0]
    except Exception:
        return u

def srcset_pick_first(srcset: str):
    try:
        first = srcset.split(",")[0].strip()
        return first.split()[0]
    except Exception:
        return None

def save_debug_html(driver, path: str):
    try:
        with open(path, "w", encoding="utf-8") as f:
            f.write(driver.page_source)
    except Exception:
        pass


In [13]:
# === 파서: 검색 결과에서 첫 링크 ===
def find_first_result_link(driver) -> str | None:
    """
    검색 결과 페이지에서 첫 번째 상품 상세 링크 추출.
    페이지 구조 변화에 대비해 여러 선택자 시도.
    """
    candidates = [
        "ul.productList li a[href]",           # 일반적인 리스트 구조
        "div.productListItem a[href]",
        "a[href*='/product/detail']",
        "a:has(img)",                          # 이미지 자식 가진 앵커
    ]
    for sel in candidates:
        try:
            els = driver.find_elements(By.CSS_SELECTOR, sel)
            if els:
                href = els[0].get_attribute("href")
                if href:
                    return href
        except Exception:
            continue
    # 정규식 폴백
    m = re.search(r'href=["\'](/product/[^"\']+)["\']', driver.page_source)
    if m:
        return urljoin(BASE_URL, html.unescape(m.group(1)))
    return None


# === 파서: 상세 페이지에서 대표 이미지 URL ===
def parse_image_from_detail(driver) -> tuple[str | None, str]:
    """
    우선순위:
      1) <meta property="og:image" content="...">
      2) 대표 이미지 <img>의 src / data-* 속성
      3) <picture><source srcset="...">
      4) 정규식: https://cf.bysuco.net/...
    """
    # 1) og:image
    try:
        ogs = driver.find_elements(By.CSS_SELECTOR, 'meta[property="og:image"]')
        for og in ogs:
            content = og.get_attribute("content")
            if content:
                return strip_query(content.strip()), "detail:meta_og:image"
    except Exception:
        pass

    # 2) 대표 이미지 후보들
    img_selectors = [
        "figure img",
        "div.productDetail img",
        "img.lazyLoadImg",
        "img[alt][src]",
        "img[src]",
    ]
    for sel in img_selectors:
        try:
            el = driver.find_elements(By.CSS_SELECTOR, sel)
            if el:
                # 첫 요소의 여러 속성 후보
                for attr in ["src", "data-src", "data-original", "data-lazy", "data-lazy-src"]:
                    v = el[0].get_attribute(attr)
                    if v and v.strip():
                        return strip_query(v.strip()), f"detail:{sel}->{attr}"
        except Exception:
            continue

    # 3) <picture><source srcset="">
    try:
        src_el = driver.find_elements(By.CSS_SELECTOR, "picture source[srcset]")
        if src_el:
            first = srcset_pick_first(src_el[0].get_attribute("srcset") or "")
            if first:
                return strip_query(first), "detail:picture>source[srcset]"
    except Exception:
        pass

    # 4) 정규식 폴백
    m = re.search(r'https://cf\.bysuco\.net/[^"\s>]+', driver.page_source)
    if m:
        return strip_query(m.group(0)), "detail:regex:cf.bysuco.net"

    return None, ""


# === 단일 이름 처리: 검색 → 첫 링크 → 상세 → 이미지 ===
def scrape_image_for_name(driver, name: str) -> tuple[str | None, str, str | None]:
    """
    return: (image_url, parse_path, fail_reason)
    """
    try:
        # 홈 방문 (쿠키/리소스 초기화)
        driver.get(BASE_URL)
        WebDriverWait(driver, NAV_TIMEOUT).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except Exception:
        pass

    # 검색 페이지 이동
    search_url = build_search_url(name)
    try:
        driver.get(search_url)
        WebDriverWait(driver, NAV_TIMEOUT).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(0.8)  # JS render 여유
    except Exception:
        if SAVE_FIRST_FAILURE_HTML:
            save_debug_html(driver, os.path.join(DEBUG_DIR, "fail_search_load.html"))
        return None, "", "request_failed(search)"

    # 첫 결과 링크
    first_link = find_first_result_link(driver)
    if not first_link:
        if SAVE_FIRST_FAILURE_HTML:
            save_debug_html(driver, os.path.join(DEBUG_DIR, "no_first_result_link.search.html"))
        return None, "no_first_result_link", "parse_failed(search_link)"

    # 상세 페이지 이동
    try:
        driver.get(first_link)
        WebDriverWait(driver, NAV_TIMEOUT).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(0.6)  # 이미지 lazy load 여유
    except Exception:
        if SAVE_FIRST_FAILURE_HTML:
            save_debug_html(driver, os.path.join(DEBUG_DIR, "fail_detail_load.html"))
        return None, "detail_request_failed", "request_failed(detail)"

    # 상세에서 이미지 추출
    img_url, path_used = parse_image_from_detail(driver)
    if not img_url:
        if SAVE_FIRST_FAILURE_HTML:
            save_debug_html(driver, os.path.join(DEBUG_DIR, "no_image_detail.html"))
        return None, path_used or "detail_no_image", "parse_failed(detail)"

    return img_url, path_used, None


In [14]:
# === Test Run: top 2 ===
df = pd.read_csv(INPUT_CSV_PATH)
if NAME_COLUMN not in df.columns:
    raise ValueError(f"`{NAME_COLUMN}` 컬럼이 없습니다.")

test_df = df.head(2).copy()
for col in ["image_url", "parse_path", "fail_reason"]:
    if col not in test_df.columns:
        test_df[col] = None

driver = make_driver(HEADLESS)

try:
    names = test_df[NAME_COLUMN].fillna("").astype(str).tolist()
    for idx, name in enumerate(names):
        nm = name.strip()
        if not nm:
            test_df.at[idx, "fail_reason"] = "empty_name"
            continue

        img_url, path_used, fail = scrape_image_for_name(driver, nm)
        test_df.at[idx, "image_url"] = img_url
        test_df.at[idx, "parse_path"] = path_used or None
        test_df.at[idx, "fail_reason"] = fail

        time.sleep(random.uniform(*CRAWL_DELAY_RANGE))
finally:
    driver.quit()

test_df.to_csv(TEST_OUTPUT_CSV_PATH, index=False)
print("Saved test:", TEST_OUTPUT_CSV_PATH)
test_df


Saved test: test_output_with_images.csv


,brand,name,size_ml,price_krw,detail_url,description,부향률,메인 어코드,탑 노트,미들 노트,베이스 노트,향 설명,image_url,parse_path,fail_reason
0,크리드,어벤투스 오 드 퍼퓸,50ml,255000.0,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...,오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향",https://cf.bysuco.net/25aa36916334ba38fde137e0...,detail:meta_og:image,None
1,크리드,어벤투스 오 드 퍼퓸,100ml,399220.0,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...,오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향",https://cf.bysuco.net/25aa36916334ba38fde137e0...,detail:meta_og:image,None


In [15]:
# === Full Run: all names ===
df_full = pd.read_csv(INPUT_CSV_PATH)
if NAME_COLUMN not in df_full.columns:
    raise ValueError(f"`{NAME_COLUMN}` 컬럼이 없습니다.")

df_full = df_full.copy()
for col in ["image_url", "parse_path", "fail_reason"]:
    if col not in df_full.columns:
        df_full[col] = None

driver = make_driver(HEADLESS)

try:
    names = df_full[NAME_COLUMN].fillna("").astype(str).tolist()
    for idx, name in enumerate(names):
        nm = name.strip()
        if not nm:
            df_full.at[idx, "fail_reason"] = "empty_name"
            continue

        img_url, path_used, fail = scrape_image_for_name(driver, nm)
        df_full.at[idx, "image_url"] = img_url
        df_full.at[idx, "parse_path"] = path_used or None
        df_full.at[idx, "fail_reason"] = fail

        time.sleep(random.uniform(*CRAWL_DELAY_RANGE))
finally:
    driver.quit()

df_full.to_csv(FULL_OUTPUT_CSV_PATH, index=False)
print("Saved full:", FULL_OUTPUT_CSV_PATH)
df_full.head(10)


Saved full: output_with_images.csv


,brand,name,size_ml,price_krw,detail_url,description,부향률,메인 어코드,탑 노트,미들 노트,베이스 노트,향 설명,image_url,parse_path,fail_reason
0,크리드,어벤투스 오 드 퍼퓸,50ml,255000.0,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...,오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향",https://cf.bysuco.net/25aa36916334ba38fde137e0...,detail:meta_og:image,None
1,크리드,어벤투스 오 드 퍼퓸,100ml,399220.0,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...,오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향",https://cf.bysuco.net/25aa36916334ba38fde137e0...,detail:meta_og:image,None
2,톰 포드,오드 우드 오 드 퍼퓸,30ml,179000.0,https://www.bysuco.com/product/show/10716,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스...,오 드 퍼퓸,"우디, 오우드, 웜 스파이시",NaN,NaN,NaN,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향,https://cf.bysuco.net/b26c9b393726d01fd533ef5c...,detail:meta_og:image,None
3,톰 포드,오드 우드 오 드 퍼퓸,50ml,249000.0,https://www.bysuco.com/product/show/10716,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스...,오 드 퍼퓸,"우디, 오우드, 웜 스파이시",NaN,NaN,NaN,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향,https://cf.bysuco.net/b26c9b393726d01fd533ef5c...,detail:meta_og:image,None
4,이솝,테싯 오 드 퍼퓸,50ml,135000.0,https://www.bysuco.com/product/show/9970,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 시트러스 / 아로마틱 / ...,오 드 퍼퓸,"시트러스, 아로마틱, 프레쉬 스파이시","유자, 시트러스",바질,"베티버, 클로브",이솝의 시그니처 향기로 따뜻하고 생기넘치며 마음을 릴렉싱 시켜주는 향,https://cf.bysuco.net/fb3db1f776b96fd2bd740154...,detail:meta_og:image,None
5,크리드,실버 마운틴 워터 오 드 퍼퓸,50ml,229000.0,https://www.bysuco.com/product/show/9368,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 시트러스 / 그린 / 프루...,오 드 퍼퓸,"시트러스, 그린, 프루티","베르가못, 만다린 오렌지","그린티, 블랙 커런트","머스크, 페티그레인, 샌달우드, 갈바넘",감귤에서 느낄 수 있는 시트러스 계열과 우디 계열의 자연스러운 조합으로 산속 깨끗한...,https://cf.bysuco.net/accec9fa9c250533933837c2...,detail:meta_og:image,None
6,크리드,실버 마운틴 워터 오 드 퍼퓸,100ml,340220.0,https://www.bysuco.com/product/show/9368,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 시트러스 / 그린 / 프루...,오 드 퍼퓸,"시트러스, 그린, 프루티","베르가못, 만다린 오렌지","그린티, 블랙 커런트","머스크, 페티그레인, 샌달우드, 갈바넘",감귤에서 느낄 수 있는 시트러스 계열과 우디 계열의 자연스러운 조합으로 산속 깨끗한...,https://cf.bysuco.net/accec9fa9c250533933837c2...,detail:meta_og:image,None
7,킬리안,문라이트 인 헤븐 오 드 퍼퓸,50ml,249000.0,https://www.bysuco.com/product/show/10520,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 스위트 / 시트러스 / 트...,오 드 퍼퓸,"스위트, 시트러스, 트로피칼","자몽, 핑크페퍼, 레몬","망고, 코코넛, 쌀","베티버, 통카빈",이국적인 매력과 자연스러움이 돋보이는 향,https://cf.bysuco.net/5eededc3a84f20475901cdb0...,detail:meta_og:image,None
8,딥티크,오 데 썽 오 드 뚜왈렛,50ml,130000.0,https://www.bysuco.com/product/show/21314,[부향률] \n- 오 드 뚜왈렛\n\n[메인 어코드]\n- 시트러스 / 플로랄 / ...,오 드 뚜왈렛,"시트러스, 플로랄, 우디","오렌지 블라썸, 비터 오렌지",주니퍼 베리,"안젤리카, 페출리",감각의 물 또는 에센스의 물로 해석되는 향,https://cf.bysuco.net/1feebb4d88474e51759d4f8f...,detail:meta_og:image,None
9,딥티크,오 데 썽 오 드 뚜왈렛,100ml,205700.0,https://www.bysuco.com/product/show/21314,[부향률] \n- 오 드 뚜왈렛\n\n[메인 어코드]\n- 시트러스 / 플로랄 / ...,오 드 뚜왈렛,"시트러스, 플로랄, 우디","오렌지 블라썸, 비터 오렌지",주니퍼 베리,"안젤리카, 페출리",감각의 물 또는 에센스의 물로 해석되는 향,https://cf.bysuco.net/1feebb4d88474e51759d4f8f...,detail:meta_og:image,None


In [16]:
import pandas as pd
import re

INPUT = "output_with_images.csv"          # 기존 파일
OUTPUT = "output_with_images_clean.csv"   # 정리본 저장 파일

df = pd.read_csv(INPUT)

def keep_only_url(s: str) -> str:
    if not isinstance(s, str):
        return s
    # 콤마 기준 첫 토큰
    first = s.split(",", 1)[0].strip().strip('"').strip("'")
    # 앞뒤 공백/컨트롤 제거
    first = re.sub(r"\s+", "", first)
    return first

if "image_url" in df.columns:
    df["image_url"] = df["image_url"].apply(keep_only_url)

df.to_csv(OUTPUT, index=False)
print("Saved:", OUTPUT)
df.head()


Saved: output_with_images_clean.csv


,brand,name,size_ml,price_krw,detail_url,description,부향률,메인 어코드,탑 노트,미들 노트,베이스 노트,향 설명,image_url,parse_path,fail_reason
0,크리드,어벤투스 오 드 퍼퓸,50ml,255000.0,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...,오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향",https://cf.bysuco.net/25aa36916334ba38fde137e0...,detail:meta_og:image,NaN
1,크리드,어벤투스 오 드 퍼퓸,100ml,399220.0,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...,오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향",https://cf.bysuco.net/25aa36916334ba38fde137e0...,detail:meta_og:image,NaN
2,톰 포드,오드 우드 오 드 퍼퓸,30ml,179000.0,https://www.bysuco.com/product/show/10716,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스...,오 드 퍼퓸,"우디, 오우드, 웜 스파이시",NaN,NaN,NaN,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향,https://cf.bysuco.net/b26c9b393726d01fd533ef5c...,detail:meta_og:image,NaN
3,톰 포드,오드 우드 오 드 퍼퓸,50ml,249000.0,https://www.bysuco.com/product/show/10716,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스...,오 드 퍼퓸,"우디, 오우드, 웜 스파이시",NaN,NaN,NaN,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향,https://cf.bysuco.net/b26c9b393726d01fd533ef5c...,detail:meta_og:image,NaN
4,이솝,테싯 오 드 퍼퓸,50ml,135000.0,https://www.bysuco.com/product/show/9970,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 시트러스 / 아로마틱 / ...,오 드 퍼퓸,"시트러스, 아로마틱, 프레쉬 스파이시","유자, 시트러스",바질,"베티버, 클로브",이솝의 시그니처 향기로 따뜻하고 생기넘치며 마음을 릴렉싱 시켜주는 향,https://cf.bysuco.net/fb3db1f776b96fd2bd740154...,detail:meta_og:image,NaN
